In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def low_rank_approximation(embedding, k):
    U, S, Vt = np.linalg.svd(embedding, full_matrices=False)
    embedding_k = U[:, :k] @ np.diag(S[:k]) @ Vt[:k, :]
    return embedding_k

In [ ]:
n = 500
d = 77
points = np.random.randn(n, d)

(500, 77)